In [1]:
train_pos_name = 'PPI_dataset/Innate_Man/train.pos.19.txt'
train_neg_name = 'PPI_dataset/Innate_Man/train.neg.19.txt'
class PPI_Data():
    def __init__(self, line, label):
        self.p1, self.p2 = line.rstrip().split(' ')
        self.label = label

PPI_pairs = []
with open(train_pos_name, 'r') as infile:
    all_lines = infile.readlines()
    for line in all_lines:
        ppi_object = PPI_Data(line, 1)
        PPI_pairs.append(ppi_object)

with open(train_neg_name, 'r') as infile:
    all_lines = infile.readlines()
    for line in all_lines:
        ppi_object = PPI_Data(line,0)
        PPI_pairs.append(ppi_object)

In [2]:
unique_protein = []
for ppi_object in PPI_pairs:
    unique_protein.append(ppi_object.p1)
    unique_protein.append(ppi_object.p2)

unique_protein = list(set(unique_protein))

In [3]:
# import requests, sys
# import json
# from tqdm import tqdm
# ppi_go_terms = {}
# remain_proteins = []
# for uniprotid in tqdm(unique_protein):
     
#     requestURL = "https://www.ebi.ac.uk/QuickGO/services/annotation/search?geneProductId="+uniprotid
#     r = requests.get(requestURL, headers={ "Accept" : "application/json"})
#     if r.ok:
#         responseBody = r.text
#         data=json.loads(responseBody)
#         for k in data['results']: 
#             go_id = k['goId'] 
#             if  uniprotid not in ppi_go_terms.keys():
#                 ppi_go_terms[uniprotid] = go_id
#             else:
#                 ppi_go_terms[uniprotid] = ppi_go_terms[uniprotid] + ';' + go_id
#         if len(data['results']) == 0:
#             remain_proteins.append(uniprotid)
#     else:
#         remain_proteins.append(uniprotid)

In [3]:

from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

In [5]:
# save_dict(ppi_go_terms, 'ppi_go_terms.pkl')

In [4]:
ppi_go_terms = load_dict('ppi_go_terms.pkl')

In [5]:
new_ppi_pairs = []
for ppi_obj in PPI_pairs:
    if ppi_obj.p1 in ppi_go_terms.keys() and ppi_obj.p2 in ppi_go_terms.keys():
        new_ppi_pairs.append(ppi_obj)

In [6]:
from Bio import SeqIO
records = list(SeqIO.parse("Uniprot_human_protein_sequences.fasta", "fasta"))

In [7]:
ppi_seqs = {}
for seq_rec in records:
    pid = seq_rec.id
    seq = str(seq_rec.seq)
    ppi_seqs[pid] = seq
     

In [8]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint

import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [9]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [10]:
import keras
feature_len = 768
max_go_len = 64
max_seq_len = 1000

prot2emb = {}
for key, value in ppi_go_terms.items():
    X_go1 =  np.zeros((1,768))
    allgos = value.split(';') 
    allgos = list(set(allgos))
    count = 0
    for  go in  allgos:
        if  os.path.exists('../ncbi_allfeatures4go/'+go+'_0.npy'):
            feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
        else:
            feature = np.random.rand(1, 768)
        if count + feature.shape[0] > max_go_len:
            break
        X_go1 = np.concatenate((X_go1,feature ))    
        count += feature.shape[0]
    prot2emb[key] =  X_go1[1:] 
protein2onehot = {}
for key, value in ppi_seqs.items():
    protein2onehot[key] =  protein_one_hot(value, max_seq_len)
        


In [11]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [12]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, Flatten
from keras.utils import multi_gpu_model
from keras.layers.merge import concatenate
from keras_radam import RAdam
from keras_lookahead import Lookahead


def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(4)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y


def build_cnn_gru_model(input_x, con_filters, gru_units):
    x = inception_block(input_x,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(gru_units, return_sequences=True))(input_x)
    x_gru = Dropout(0.3)(x_gru)
     
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_c = Attention()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    x_gru_c = Attention()(x_gru)
    x = Concatenate()([x_a, x_b, x_c, x_gru_a, x_gru_b,   x_gru_c])
    x = Dense(256,activation='relu')(x)
    return x



def build_model():
    con_filters = 256
    gru_units = 64
    left_input_go = Input(shape=(max_go_len,feature_len))
    right_input_go = Input(shape=(max_go_len,feature_len))
    
    
    left_input_seq = Input(shape=(max_seq_len,20))
    right_input_seq = Input(shape=(max_seq_len,20))
    
     
 
     
    left_x_go = build_cnn_gru_model(left_input_go, con_filters, gru_units)
    right_x_go = build_cnn_gru_model(right_input_go, con_filters,gru_units)
    
    left_x_seq = build_cnn_gru_model(left_input_seq, con_filters//4, gru_units)
    right_x_seq = build_cnn_gru_model(right_input_seq, con_filters//4, gru_units)
     
    
   
    x =   Concatenate()([left_x_go  , right_x_go, left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
  
     
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_go, right_input_go, left_input_seq, right_input_seq], output)
#     model = multi_gpu_model(model, gpus=2)
    optimizer = Lookahead(RAdam())

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


model = build_model()
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 768)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 64, 768)      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1000, 20)     0                                            
___________________________________________________________________________

In [30]:
# save_model_name = '../SC_CV/sc_GoplusSeq0-0.hdf5'
# model.load_weights(save_model_name)

In [13]:
X_go_left = np.zeros((len(new_ppi_pairs), max_go_len, 768))
X_seq_left = np.zeros((len(new_ppi_pairs), max_seq_len, 20))

X_go_right = np.zeros((len(new_ppi_pairs), max_go_len, 768))
X_seq_right = np.zeros((len(new_ppi_pairs), max_seq_len, 20))

y = np.zeros((len(new_ppi_pairs),1))
for i, ppi_obj in enumerate(new_ppi_pairs):
    p1 = ppi_obj.p1
    p2 = ppi_obj.p2
    y[i] = ppi_obj.label
    feature_left = prot2emb[p1]
    X_go_left[i,:feature_left.shape[0]] = feature_left
    
    feature_right = prot2emb[p2]
    X_go_right[i,:feature_right.shape[0]] = feature_right
    
    
    X_seq_left[i] = protein2onehot[p1]
    X_seq_right[i] = protein2onehot[p2]
    
 

In [14]:
save_model_name = 'Innate_Man_19_2'
earlyStopping = EarlyStopping(monitor='val_acc', patience=20, verbose=0, mode='max')
save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_acc', mode='max', save_weights_only=True)

model.fit([X_go_left, X_go_right, X_seq_left, X_seq_right], y, epochs=100, batch_size=128, validation_split=0.1,callbacks=[earlyStopping, save_checkpoint])




Train on 6796 samples, validate on 756 samples
Epoch 1/100
6796/6796 [==============================] - 31s 5ms/step - loss: 0.5573 - acc: 0.6979 - val_loss: 0.6903 - val_acc: 0.6243
Epoch 2/100
6796/6796 [==============================] - 19s 3ms/step - loss: 0.3534 - acc: 0.8451 - val_loss: 0.6977 - val_acc: 0.6640
Epoch 3/100
6796/6796 [==============================] - 19s 3ms/step - loss: 0.3003 - acc: 0.8696 - val_loss: 0.3189 - val_acc: 0.8624
Epoch 4/100
6796/6796 [==============================] - 18s 3ms/step - loss: 0.2302 - acc: 0.9094 - val_loss: 1.0518 - val_acc: 0.5516
Epoch 5/100
6796/6796 [==============================] - 18s 3ms/step - loss: 0.2154 - acc: 0.9161 - val_loss: 1.3873 - val_acc: 0.3598
Epoch 6/100
6796/6796 [==============================] - 19s 3ms/step - loss: 0.2006 - acc: 0.9211 - val_loss: 0.6313 - val_acc: 0.7077
Epoch 7/100
6796/6796 [==============================] - 19s 3ms/step - loss: 0.1538 - acc: 0.9382 - val_loss: 0.8969 - val_acc: 0.6111
E

In [15]:
test_PPI_pairs = []
all_test_protein = []
with open('PPI_dataset/Innate_Man/test.pos.c1.19.txt', 'r') as infile:
    all_lines = infile.readlines()
    for line in all_lines:
        p1,p2 = line.rstrip().split(' ')
        ppi_obj = PPI_Data(line,1)
        test_PPI_pairs.append(ppi_obj)
        all_test_protein.append(p1)
        all_test_protein.append(p2)
with open('PPI_dataset/Innate_Man/test.neg.c1.19.txt', 'r') as infile:
    all_lines = infile.readlines()
    for line in all_lines:
        p1,p2 = line.rstrip().split(' ')
        ppi_obj = PPI_Data(line,0)
        test_PPI_pairs.append(ppi_obj)
        all_test_protein.append(p1)
        all_test_protein.append(p2)
    
        
        


In [7]:
# unique_test_protein = list(set(all_test_protein))

In [9]:
# import requests, sys
# import json
# from tqdm import tqdm
# ppi_go_terms = {}
# remain_proteins = []
# for uniprotid in tqdm(unique_test_protein):
     
#     requestURL = "https://www.ebi.ac.uk/QuickGO/services/annotation/search?geneProductId="+uniprotid
#     r = requests.get(requestURL, headers={ "Accept" : "application/json"})
#     if r.ok:
#         responseBody = r.text
#         data=json.loads(responseBody)
#         for k in data['results']: 
#             go_id = k['goId'] 
#             if  uniprotid not in ppi_go_terms.keys():
#                 ppi_go_terms[uniprotid] = go_id
#             else:
#                 ppi_go_terms[uniprotid] = ppi_go_terms[uniprotid] + ';' + go_id
#         if len(data['results']) == 0:
#             remain_proteins.append(uniprotid)
#     else:
#         remain_proteins.append(uniprotid)

100%|██████████| 1198/1198 [25:02<00:00,  1.25s/it]


In [12]:
# save_dict(ppi_go_terms, 'ppi_goterms_test.pkl')

In [16]:
ppi_go_terms = load_dict('ppi_goterms_test.pkl')
ppi_disease = load_dict('ppi_disease_terms_test.pkl')

In [5]:
# import requests, sys
# import json
# from tqdm import tqdm
# ppi_disease_terms = {}
# remain_proteins = []
# for uniprotid in tqdm(ppi_go_terms.keys()):
     
#     requestURL = "https://www.disgenet.org/api/gda/gene/uniprot/"+uniprotid+"?type=disease&format=json"
#     r = requests.get(requestURL, headers={ "Accept" : "application/json"})
#     if r.ok:
#         responseBody = r.text
#         data=json.loads(responseBody)
#         for k in  data: 
#             go_id = k['diseaseid'] 
#             if  uniprotid not in ppi_disease_terms.keys():
#                 ppi_disease_terms[uniprotid] = go_id
#             else:
#                 ppi_disease_terms[uniprotid] = ppi_disease_terms[uniprotid] + ';' + go_id
#         if len(data) == 0:
#             remain_proteins.append(uniprotid)
#     else:
#         remain_proteins.append(uniprotid)

100%|██████████| 1156/1156 [53:59<00:00,  2.80s/it] 


In [6]:
# save_dict(ppi_disease_terms, 'ppi_disease_terms_test.pkl')

In [17]:
new_test_PPI_pairs = []
for ppi_obj in test_PPI_pairs:
    if ppi_obj.p1 in ppi_disease.keys() and ppi_obj.p2 in ppi_disease.keys() and ppi_obj.p1 in ppi_go_terms.keys() and ppi_obj.p2 in ppi_go_terms.keys():
        new_test_PPI_pairs.append(ppi_obj)

In [18]:
print(len(new_ppi_pairs), len(new_test_PPI_pairs))

7552 575


In [19]:
all_prots = []
pos_num = 0
neg_num = 0
for ppi_obj in new_ppi_pairs:
    all_prots.append(ppi_obj.p1)
    all_prots.append(ppi_obj.p2)
    if ppi_obj.label == 1:
        pos_num += 1
    else:
        neg_num += 1

for ppi_obj in new_test_PPI_pairs:
    all_prots.append(ppi_obj.p1)
    all_prots.append(ppi_obj.p2)
    if ppi_obj.label == 1:
        pos_num += 1
    else:
        neg_num += 1
print(len(set(all_prots)), pos_num,neg_num )

6841 4352 3775


In [20]:
# compute jaccard_similarity
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [21]:
pos_jaccard = []
neg_jaccard = []
for ppi_obj in new_test_PPI_pairs:
 
    p1_disease = ppi_disease[ppi_obj.p1].split(';')
    p2_disease = ppi_disease[ppi_obj.p2].split(';')
    jaccard_s = jaccard_similarity(p1_disease, p2_disease)
    if ppi_obj.label == 1:
        pos_jaccard.append(jaccard_s)
    else:
        neg_jaccard.append(jaccard_s)
        
    

In [22]:
import numpy as np
pos_jaccard = np.array(pos_jaccard)
neg_jaccard = np.array(neg_jaccard)
print(np.mean(pos_jaccard), np.mean(neg_jaccard))

0.05779666112444239 0.017382196855757384


In [26]:
feature_len = 768
max_go_len = 64
max_seq_len = 1000

prot2emb = {}
for key, value in ppi_go_terms.items():
    X_go1 =  np.zeros((1,768))
    allgos = value.split(';') 
    allgos = list(set(allgos))
    count = 0
    for  go in  allgos:
        if  os.path.exists('../ncbi_allfeatures4go/'+go+'_0.npy'):
            feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')[1:-1]
        else:
            feature = np.random.rand(1, 768)
        if count + feature.shape[0] > max_go_len:
            break
        X_go1 = np.concatenate((X_go1,feature ))    
        count += feature.shape[0]
    prot2emb[key] =  X_go1[1:] 
 

In [27]:
X_go_left = np.zeros((len(new_test_PPI_pairs), max_go_len, 768))
X_seq_left = np.zeros((len(new_test_PPI_pairs), max_seq_len, 20))

X_go_right = np.zeros((len(new_test_PPI_pairs), max_go_len, 768))
X_seq_right = np.zeros((len(new_test_PPI_pairs), max_seq_len, 20))

y = np.zeros((len(new_test_PPI_pairs),1))
for i, ppi_obj in enumerate(new_test_PPI_pairs):
    p1 = ppi_obj.p1
    p2 = ppi_obj.p2
    y[i] = ppi_obj.label
    feature_left = prot2emb[p1]
    X_go_left[i,:feature_left.shape[0]] = feature_left
    
    feature_right = prot2emb[p2]
    X_go_right[i,:feature_right.shape[0]] = feature_right
    
    
    X_seq_left[i] = protein2onehot[p1]
    X_seq_right[i] = protein2onehot[p2]
    
 

In [28]:
model.load_weights('Innate_Man_19_2')
# model.load_weights('../SC_CV/sc_GoplusSeq1-0.hdf5')
y_pred_prob = model.predict([X_go_left,X_go_right, X_seq_left, X_seq_right ], batch_size=128)

In [29]:
from sklearn.metrics import accuracy_score
y_pred = (y_pred_prob > 0.5)
acc = accuracy_score(y, y_pred)
print(acc)

0.9078260869565218


In [30]:
ind = np.argsort(y_pred_prob, axis=0)

In [31]:
# select worst 1%
top20_len = 20 #int(np.floor(y_pred_prob.shape[0]*0.2))

In [32]:
worst_top20_ind = ind[:top20_len]
best_top20_ind = ind[y_pred_prob.shape[0]-top20_len:]

In [59]:
with open('predict_prob.txt', 'w') as wfile:
    for i in range(ind.shape[0]):
        index = ind[i]
        ppi_obj = new_test_PPI_pairs[index[0]]
        p1_disease = ppi_disease[ppi_obj.p1].split(';')
        p2_disease = ppi_disease[ppi_obj.p2].split(';')
        intersection = len(list(set(p1_disease).intersection(p2_disease)))
        is_common = intersection > 0
        wfile.writelines(ppi_obj.p1+';'+ppi_obj.p2+';'+str(len(p1_disease)) +';'+ str(len(p2_disease)) +';'+ str(ppi_obj.label)+';'+ str(y_pred_prob[index][0])+';'+str(intersection)+';'+str(is_common)+'\n')

In [ ]:
mean_probs = []

In [46]:

top20_len = 250
worst_top20_ind = ind[:top20_len]
best_top20_ind = ind[y_pred_prob.shape[0]-top20_len:]
best_top20_jaccard = 0
for bi in best_top20_ind:
    ppi_obj = new_test_PPI_pairs[bi[0]]
    p1_disease = ppi_disease[ppi_obj.p1].split(';')
    p2_disease = ppi_disease[ppi_obj.p2].split(';')
    intersection = len(list(set(p1_disease).intersection(p2_disease)))
    if intersection > 0:
        best_top20_jaccard += 1
    
print(np.mean(y_pred_prob[best_top20_ind]))
print(best_top20_jaccard/top20_len)
print(best_top20_jaccard)
    
#     else:
#         print(ppi_obj.p1, ppi_obj.p2)



0.98517776
0.828
207


19

In [35]:
print(best_top20_jaccard)

9


In [56]:
top20_len = 5
worst_top20_ind = ind[:top20_len]

worst_top20_jaccard = 0
for bi in worst_top20_ind:
    ppi_obj = new_test_PPI_pairs[bi[0]]
    p1_disease = ppi_disease[ppi_obj.p1].split(';')
    p2_disease = ppi_disease[ppi_obj.p2].split(';')
    intersection = len(list(set(p1_disease).intersection(p2_disease)))
    if intersection > 0:
        worst_top20_jaccard += 1

print(np.mean(y_pred_prob[worst_top20_ind]))
print(worst_top20_jaccard/top20_len)
print(worst_top20_jaccard)
#     else:
#         print(ppi_obj.p1, ppi_obj.p2)
#     worst_top10_jaccard += jaccard_similarity(p1_disease, p2_disease)
        
#     best_top10_jaccard += jaccard_similarity(p1_disease, p2_disease)
# print(best_top20_jaccard, worst_top20_jaccard)

2.0980835e-05
0.2
1


In [58]:
mean_probs = [0.9995886, 0.99943006, 0.9991978, 0.998612, 0.9972, 0.99172 , 0.985177 , 0.114573, 0.02742,
              0.001659 , 0.000433, 9.042323e-05, 4.4602155e-05 , 2.0980835e-05]
ppcds = [1, 0.9, 0.95, 0.84, 0.86, 0.82, 0.828, 0.424, 0.375, 0.29, 0.28, 0.2, 0.3, 0.2]
import scipy
from scipy.stats import pearsonr

 
r_row, p_value = pearsonr(mean_probs, ppcds)
print(r_row, p_value)


0.9799621269004444 8.951176678778831e-10


In [36]:
print(best_top20_jaccard, y.shape[0]*0.2,  y.shape[0])

18 115.0 575


In [53]:
worst_top10_jaccard = 0
for bi in worst_top10_ind:
    ppi_obj = new_test_PPI_pairs[bi[0]]
    p1_disease = ppi_disease[ppi_obj.p1].split(';')
    p2_disease = ppi_disease[ppi_obj.p2].split(';')
    worst_top10_jaccard += jaccard_similarity(p1_disease, p2_disease)

In [58]:
print(best_top10_jaccard/top10_len, worst_top10_jaccard/top10_len)

0.056705748937831424 0.011323716207573458


In [55]:
top20_len = int(np.floor(y_pred_prob.shape[0]*0.2))
worst_top20_ind = ind[:top20_len]
best_top20_ind = ind[y_pred_prob.shape[0]-top20_len:]

In [56]:
best_top20_jaccard = 0
for bi in best_top20_ind:
    ppi_obj = new_test_PPI_pairs[bi[0]]
    p1_disease = ppi_disease[ppi_obj.p1].split(';')
    p2_disease = ppi_disease[ppi_obj.p2].split(';')
    best_top20_jaccard += jaccard_similarity(p1_disease, p2_disease)
worst_top20_jaccard = 0
for bi in worst_top20_ind:
    ppi_obj = new_test_PPI_pairs[bi[0]]
    p1_disease = ppi_disease[ppi_obj.p1].split(';')
    p2_disease = ppi_disease[ppi_obj.p2].split(';')
    worst_top20_jaccard += jaccard_similarity(p1_disease, p2_disease)

In [59]:
print(best_top20_jaccard/top20_len, worst_top20_jaccard/top20_len)

0.06034204469281473 0.011811925199367117


In [60]:
top30_len = int(np.floor(y_pred_prob.shape[0]*0.3))
worst_top30_ind = ind[:top30_len]
best_top30_ind = ind[y_pred_prob.shape[0]-top30_len:]
best_top30_jaccard = 0
for bi in best_top30_ind:
    ppi_obj = new_test_PPI_pairs[bi[0]]
    p1_disease = ppi_disease[ppi_obj.p1].split(';')
    p2_disease = ppi_disease[ppi_obj.p2].split(';')
    best_top30_jaccard += jaccard_similarity(p1_disease, p2_disease)
worst_top30_jaccard = 0
for bi in worst_top30_ind:
    ppi_obj = new_test_PPI_pairs[bi[0]]
    p1_disease = ppi_disease[ppi_obj.p1].split(';')
    p2_disease = ppi_disease[ppi_obj.p2].split(';')
    worst_top30_jaccard += jaccard_similarity(p1_disease, p2_disease)

In [61]:
print(best_top30_jaccard/top30_len, worst_top30_jaccard/top30_len)

0.06038744522127595 0.015712698183245187
